# Импорт библиотек и обучающих данных:


---

In [1]:
import numpy as np
import pandas as pd
import zipfile
from matplotlib import pyplot as plt
import shutil 
from tqdm import tqdm
import torch
import torchvision
import time
import copy
from torchvision import transforms, models
import os
import torch.nn as nn

In [2]:
torch.cuda.is_available()

True

In [3]:
стоп

NameError: name 'стоп' is not defined

# Разархивация изображений

---

In [ ]:
# with zipfile.ZipFile(r"data\welds.zip",'r') as zip_obj:
#         zip_obj.extractall(r"data\binary_root")
data_root = r"C:\Users\HP\Documents\Artificial_Intelligence\Проекты для CV\weld_inspection\data\binary_root\al5083\train"
print('Файлы выгрузились из архива успешно!') 

# Разметка данных:

---

У меня есть json файл с разметкой. Надо его пропарсить. Добавляю столбец для класса в случае многоклассовой и бинарной классификации

In [ ]:
labels = pd.read_json(r"C:\Users\HP\Documents\Artificial_Intelligence\Проекты для CV\weld_inspection\data\binary_root\al5083\train\train.json", typ='series')
labels = labels.to_frame().reset_index().rename(columns={'index':'path',0:'class'})
labels['class'] = labels['class'].replace({0:'good_weld',
                                           1:'burn_through',
                                           2:'contamination',
                                           3:'lack_of_fusion',
                                           4:'misalignment',
                                           5:'lack_of_penetration'})
labels = labels.sort_values(by='class').reset_index().drop('index',axis=1)
labels = labels.loc[labels['class'] != 'good_weld']
classes = labels['class'].unique()
labels

Для повышения точности модели решено создать две нейросети - первая будет бинарно классифицировать хорошие и плохие швы, а вторая будет определять вид дефекта у плохих швов. Для балансировки данных в бинарной классификации необходимо удвоить количество хороших швов за счёт аугментации.

# Разбивка данных на обучающие и валидационные:

---



Здесь принимаю первое решение: размер валидационной выборки. Он зависит от размера датасета и должен быть случайным. Я хочу использовать порядка 16% датасета для валидации. Валидация случайная - каждый 6-ой файл.

In [ ]:
many_classes_root = r"data\many_classes_classifier"
train_dir = os.path.join(many_classes_root,'train')
val_dir = os.path.join(many_classes_root,'val')
class_names = classes # некий список классов нашей классификации.

for dir_name in [train_dir, val_dir]: #итерация через 2 строчки
    for class_name in class_names: # для каждого имени класса  в списке классов
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)
print('Успешно созданы папки val и train с классами швов!')

Столкнулся с проблемой: при копировании файлов в общую директорию, файлы с одинаковым именем заменяют друг друга.. Надо их как-то переименовывать

In [ ]:
class_names = classes
data_root =r"C:\Users\HP\Documents\Artificial_Intelligence\Проекты для CV\weld_inspection\data\binary_root\al5083\train"
for class_name in class_names: 
    for i, file_name in enumerate(tqdm(labels['path'].loc[labels['class']==class_name].tolist())):
        pic_name = str(i) + '.png'
        if i % 6 != 0:
            shutil.copy(os.path.join(data_root, file_name), os.path.join(os.path.join(train_dir, class_name, pic_name)))
        else:
            shutil.copy(os.path.join(data_root, file_name), os.path.join(os.path.join(val_dir, class_name, pic_name)))
print('Изображения отсортированы по train и val!') 

Проверяем корректность разбивки файлов:

In [ ]:
lst_train = []
lst_val = []
for type in classes:
    items_train = os.listdir(os.path.join(train_dir,type))
    items_val =  os.listdir(os.path.join(val_dir,type))
    lst_train.append(len(items_train))
    lst_val.append(len(items_val))
print('Всего изображений в деле: ', sum(lst_train) + sum(lst_val))
print('Всего на обучении: ', sum(lst_train))
print('Всего на валидации: ', sum(lst_val))
print('Отношение обучающих данных к валидационным: ', sum(lst_train) / sum(lst_val))
root = r"data\many_classes_classifier\train"
for cls in classes:
    print(cls, len(os.listdir(os.path.join(root,cls))))

In [ ]:
print(1485 *2 * 1.7)
print(5270)
print(3356 * 1.5)
print(2349 * 2.2)
print(2460 * 2)

# Трансформация и аугментация

In [ ]:
import os
import random
from PIL import Image
# Путь к директории с изображениями
image_dir = r"data\many_classes_classifier\val\burn_through"

# Получение списка файлов изображений в директории
image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f)) and f.endswith(('.jpg', '.jpeg', '.png'))]
file_list = os.listdir(image_dir)
random.shuffle(file_list)
num_images = len(file_list)
num_images_70_percent = int(0.7 * num_images)

# Перемешивание списка файлов

# Итерация по первым 70% файлов
for i in tqdm(range(num_images_70_percent)):
    file_name = file_list[i]
    image_path = os.path.join(image_dir, file_name)
    image = Image.open(image_path)
    brightness_augmentation = transforms.ColorJitter(brightness=0.4)
    augmented_image = brightness_augmentation(image)
    flipped_file_name = "brightness" + file_name
    flipped_file_path = os.path.join(image_dir, flipped_file_name)
    augmented_image.save(flipped_file_path)
    
# Создание зеркальных копий изображений
# for file_name in tqdm(image_files):
#     file_path = os.path.join(image_dir, file_name)
#     image = Image.open(file_path)
#     flipped_image = image.transpose(Image.FLIP_LEFT_RIGHT)
#     flipped_file_name = "flipped_" + file_name
#     flipped_file_path = os.path.join(image_dir, flipped_file_name)
#     flipped_image.save(flipped_file_path)

In [ ]:
image_dir = r"data\many_classes_classifier\val\lack_of_fusion"

# Получение списка файлов изображений в директории
image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f)) and f.endswith(('.jpg', '.jpeg', '.png'))]
file_list = os.listdir(image_dir)
random.shuffle(file_list)
num_images = len(file_list)
num_images_50_percent = int(0.5 * num_images)

# Перемешивание списка файлов

# Итерация по первым 50% файлов
for i in tqdm(range(num_images_50_percent)):
    file_name = file_list[i]
    image_path = os.path.join(image_dir, file_name)
    image = Image.open(image_path)
    brightness_augmentation = transforms.ColorJitter(brightness=0.4)
    augmented_image = brightness_augmentation(image)
    flipped_file_name = "brightness" + file_name
    flipped_file_path = os.path.join(image_dir, flipped_file_name)
    augmented_image.save(flipped_file_path)
    
# Создание зеркальных копий изображений
# for file_name in tqdm(range(num_images_70_percent)):
#     file_path = os.path.join(image_dir, file_name)
#     image = Image.open(file_path)
#     flipped_image = image.transpose(Image.FLIP_LEFT_RIGHT)
#     flipped_file_name = "flipped_" + file_name
#     flipped_file_path = os.path.join(image_dir, flipped_file_name)
#     flipped_image.save(flipped_file_path)

In [ ]:
image_dir = r"data\many_classes_classifier\val\lack_of_penetration"

# Получение списка файлов изображений в директории
image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f)) and f.endswith(('.jpg', '.jpeg', '.png'))]
file_list = os.listdir(image_dir)
random.shuffle(file_list)
num_images = len(file_list)
num_images_20_percent = int(0.2 * num_images)

# Перемешивание списка файлов

# Итерация по первым 20% файлов
for i in tqdm(range(num_images_20_percent)):
    file_name = file_list[i]
    image_path = os.path.join(image_dir, file_name)
    image = Image.open(image_path)
    brightness_augmentation = transforms.ColorJitter(brightness=0.4)
    augmented_image = brightness_augmentation(image)
    flipped_file_name = "brightness" + file_name
    flipped_file_path = os.path.join(image_dir, flipped_file_name)
    augmented_image.save(flipped_file_path)
    
# Создание зеркальных копий изображений
# for file_name in tqdm(image_files):
#     file_path = os.path.join(image_dir, file_name)
#     image = Image.open(file_path)
#     flipped_image = image.transpose(Image.FLIP_LEFT_RIGHT)
#     flipped_file_name = "flipped_" + file_name
#     flipped_file_path = os.path.join(image_dir, flipped_file_name)
#     flipped_image.save(flipped_file_path)

In [ ]:
image_dir = r"data\many_classes_classifier\val\misalignment"

# Получение списка файлов изображений в директории
image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f)) and f.endswith(('.jpg', '.jpeg', '.png'))]
file_list = os.listdir(image_dir)
# random.shuffle(file_list)
# num_images = len(file_list)
# num_images_70_percent = int(0.7 * num_images)

# Перемешивание списка файлов

# Итерация по первым 70% файлов
# for i in tqdm(range(num_images_70_percent)):
#     file_name = file_list[i]
#     image_path = os.path.join(image_dir, file_name)
#     image = Image.open(image_path)
#     brightness_augmentation = transforms.ColorJitter(brightness=0.4)
#     augmented_image = brightness_augmentation(image)
#     flipped_file_name = "brightness" + file_name
#     flipped_file_path = os.path.join(image_dir, flipped_file_name)
#     augmented_image.save(flipped_file_path)
    
# Создание зеркальных копий изображений
for file_name in tqdm(image_files):
    file_path = os.path.join(image_dir, file_name)
    image = Image.open(file_path)
    flipped_image = image.transpose(Image.FLIP_LEFT_RIGHT)
    flipped_file_name = "flipped_" + file_name
    flipped_file_path = os.path.join(image_dir, flipped_file_name)
    flipped_image.save(flipped_file_path)

В качестве baseline'а применяется архитектура VGG-19. У данной архитектуры есть требования к исходным изображениям:

По этой причине трансформация валидационного изображения будет выглядеть следующим образом:

In [4]:
many_classes_root = r"data\many_classes_classifier"
train_dir = os.path.join(many_classes_root,'train')
val_dir = os.path.join(many_classes_root,'val')
val_transforms = transforms.Compose([ # трансформации для валидационных изображений:
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)

Для обучающих изображений будут применены следующие аугментации:

In [5]:
t = 0.0
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.RandomHorizontalFlip(), 
    #transforms.RandomRotation(90),
#     transforms.ColorJitter(brightness=t, contrast=t, saturation=t, hue=t),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
])
train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms)
# t = 0.1
# train_transforms = transforms.Compose([# дальше идут трансформации всех изображений для обучения, а именно:
#     transforms.Resize((224, 224)),
#     transforms.RandomHorizontalFlip(), #рандомно выполняет "переворот" изображения
#     transforms.RandomRotation(90),
#     transforms.ColorJitter(brightness=t, contrast=t, saturation=t, hue=t),
#     transforms.ToTensor(), # преобразует изображение в тензор
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # нормализация для ускорения обучения (среднее = 0, отклонение =1) 
# ])
# train_dataset += torchvision.datasets.ImageFolder(train_dir, train_transforms)
# t = 0.2
# train_transforms = transforms.Compose([# дальше идут трансформации всех изображений для обучения, а именно:
#     transforms.Resize((224, 224)),
#     transforms.RandomHorizontalFlip(), #рандомно выполняет "переворот" изображения
#     transforms.RandomRotation(90),
#     transforms.ColorJitter(brightness=t, contrast=t, saturation=t, hue=t),
#     transforms.ToTensor(), # преобразует изображение в тензор
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # нормализация для ускорения обучения (среднее = 0, отклонение =1) 
# ])
# train_dataset += torchvision.datasets.ImageFolder(train_dir, train_transforms)
# t = 0.3
# train_transforms = transforms.Compose([# дальше идут трансформации всех изображений для обучения, а именно:
#     transforms.Resize((224, 224)),
#     transforms.RandomHorizontalFlip(), #рандомно выполняет "переворот" изображения
#     transforms.RandomRotation(90),
#     transforms.ColorJitter(brightness=t, contrast=t, saturation=t, hue=t),
#     transforms.ToTensor(), # преобразует изображение в тензор
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # нормализация для ускорения обучения (среднее = 0, отклонение =1) 
# ])
# train_dataset += torchvision.datasets.ImageFolder(train_dir, train_transforms)

In [6]:
print('Обучающих изображений: ', len(train_dataset))
print('Валидационных изображений: ', len(val_dataset))

Обучающих изображений:  24947
Валидационных изображений:  5190


In [7]:
len(train_dataset) / len(val_dataset)

4.806743737957611

# Определяю размер batches

При большом числе будет меньше обновлений весов - модель попробует учесть все изображения в одном разе. При малом числе батчей - веса будут обновляться постоянно, но модель будет учитывать малое число изображений. Начнём с 8, буду исследовать для:

In [8]:
batch_size = 8 # размер бача, т.е. кол-во пикчей, используемых на 1 эпохе

In [9]:
# torch.set_default_dtype(torch.float32)

Применение трансформаций для изображений:

In [10]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=2) 

# Определение архитектуры модели:

Архитектура исследуемой модели представлена ниже:

In [11]:
# Загрузка предобученных моделей
# model = models.vgg11(pretrained=True)
#model = models.vgg19(pretrained=True)
#model = models.alexnet(pretrained=True)
model = models.densenet169(pretrained=True) 
#model = models.resnet18(pretrained=True)
#model = models.resnet50(pretrained=True)

# Замена последнего полносвязного слоя на новый слой для 5 классов
num_classes = 5
model.classifier._modules['6'] = torch.nn.Linear(4096, num_classes)
# Замена последнего полносвязного слоя на новый слой для 6 классов ДЛЯ RESNET
# num_classes = 6
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, num_classes)
# Вывод архитектуры модели с новым слоем
print('Всё ок')

C:\Users\HP\anaconda3\envs\gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HP\anaconda3\envs\gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet169_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet169_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Всё ок


In [12]:
device = torch.device("cuda:0")
model = model.to(device)

Ниже определяем лосс-функцию. По дефолту это Кросс-энтропия, т.к в примере была именно она

In [13]:
criterion = torch.nn.CrossEntropyLoss() # функция потерь

Learning_rate = 0.001 по дефолту

In [14]:
learning_rate = 0.001

Оптимайзер в начале исследования всегда лучше начинать с СГД:

In [15]:
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# beta1 = 0.9
# beta2 = 0.999
# eps = 0.00001
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(beta1, beta2), eps=eps, amsgrad = True)
optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Rmspror(model.parameters(), lr=learning_rate)
# alpha = 0.01
# optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, alpha=alpha)

Хочу отметить, что на этом этапе я усвоил урок - параметры оптимизатора - очень важны. Поменяв alpha для RMprop с 0.9 до 0.01 результат изменился в 3,5 раза.

Уменьшение шага градиента сначала поставим 7

In [16]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # уменьшение шага градиентного спуска

Число эпох 15, посмотрим как долго будет обучаться модель:

In [17]:
num_epochs=50

In [18]:
device = torch.device("cuda:0")
model = model.to(device)

# Обучение модели:

In [19]:
best_accuracy = 0
no_improvement_count = 0
patience = 5
for epoch in range(50):
    # Обучение на тренировочных данных
    model.train()
    for inputs, labels in tqdm(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # Оценка точности на валидационных данных
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        for inputs, labels in tqdm(val_dataloader):
          inputs = inputs.to(device)
          labels = labels.to(device)
          outputs = model(inputs)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
        accuracy = correct / total
    
    # Проверяем, улучшилась ли точность на этой эпохе
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        no_improvement_count = 0
        print('Модель улучшена!')
    else:
        no_improvement_count += 1
        print('Модель не улучшилась, обучение продолжается')
    
    # Если точность не улучшалась в течение patience эпох, прекращаем обучение
    if no_improvement_count == patience:
        print('No improvement for {} epochs. Stopping training.'.format(patience))
        break
    
    # Выводим информацию о текущей эпохе
    print('Epoch {}/{}:'.format(epoch + 1, num_epochs), flush=True)
    print('Training Loss: {:.4f}'.format(loss.item()), flush=True)
    print('Validation Accuracy: {:.4f}'.format(accuracy), flush=True)

 45%|████▌     | 1419/3119 [12:40<15:11,  1.87it/s]


KeyboardInterrupt: 

# Сохранение обученной модели:

In [ ]:
torch.save(best_model, 'desnet169_adagrad_5classes.pth')

In [ ]:
best_model

In [ ]:
params = best_model.state_dict()

# Вывод весов модели
for name, weights in params.items():
    print(name, weights)